<a href="https://colab.research.google.com/github/AntonyLeons/Data-Mining-ACW/blob/master/Legacy_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Legacy Data

In [0]:
import pandas as pd
import numpy as np
import random # import libs
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))